In [31]:
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as ET

In [32]:
# 1. Chargement des Hôtes (XML)
tree = ET.parse('olympic_hosts.xml')
root = tree.getroot()

hosts_data = []
for row in root.findall('row'):
    hosts_data.append({
        'slug_game': row.find('game_slug').text,
        'game_year': int(row.find('game_year').text),
        'game_season': row.find('game_season').text,
        'host_country': row.find('game_location').text
    })
df_hosts = pd.DataFrame(hosts_data)

In [33]:
# 2. Chargement des Résultats (HTML)
# Note: read_html retourne une liste de DataFrames, on prend le premier
print("Chargement du fichier HTML (cela peut prendre quelques secondes)...")
dfs_html = pd.read_html('olympic_results.html')
df_results = dfs_html[0]

Chargement du fichier HTML (cela peut prendre quelques secondes)...


In [34]:
# 3. Chargement des Athlètes (JSON)
with open('olympic_athletes.json', 'r') as f:
    athletes_data = json.load(f)
df_athletes = pd.DataFrame(athletes_data)

In [35]:
# creation have bio (annuler)
df_athletes['have_bio'] = df_athletes['bio'].apply(
    lambda x: 1 if x is not None and len(str(x).strip()) > 0 else 0
)

In [36]:
# fusion
df_main = pd.merge(df_results, df_hosts, on='slug_game', how='left')

In [37]:
# creation have bio (utilisation annuler)
df_main = pd.merge(df_main, df_athletes[['athlete_url', 'have_bio']], on='athlete_url', how='left')

In [38]:
# Remplir les NaN de have_bio par 0 (utilisation annuler)
df_main['have_bio'] = df_main['have_bio'].fillna(0).astype(int)

In [39]:
# Remplacer les NaN dans medal_type par "No Medal"
df_main['medal_type'] = df_main['medal_type'].fillna('No Medal')

In [40]:
# harmonisation code pays
df_main['country_code'] = df_main['country_code'].astype(str).str.strip()

In [41]:
# creation is host
df_main['is_host'] = df_main.apply(lambda x: 1 if str(x['country_name']) in str(x['host_country']) else 0, axis=1)

In [42]:
# encodage medaille
medal_mapping = {'GOLD': 3, 'SILVER': 2, 'BRONZE': 1, 'No Medal': 0}
df_main['medal_value'] = df_main['medal_type'].map(medal_mapping)

In [43]:


#CONSTRUIRE LES FEATUREs

#  Nombre de médailles par Pays/Sport/Année.
df_target = df_main.groupby(['country_name', 'discipline_title', 'game_year', 'game_season', 'is_host'])['medal_value'].sum().reset_index()
df_target.rename(columns={'medal_value': 'medals_won'}, inplace=True)

# Nombre d'athletes
df_delegation = df_main.groupby(['country_name', 'discipline_title', 'game_year', 'game_season']).size().reset_index(name='delegation_size')

# Potentiel du Sport (Nombre TOTAL de medailles distribuees dans ce sport sur cette annee)
# Cela aide le modèle à comprendre que la Natation permet de gagner beaucoup plus de points que le Handball.
df_sport_scale = df_main.groupby(['discipline_title', 'game_year', 'game_season'])['medal_value'].sum().reset_index(name='total_medals_available_in_sport')

# Puissance Globale du Pays (Total medailles du pays tous sports confondus cette sur cette annee)
# On va l'utiliser pour créer un LAG (historique)
df_country_total = df_main.groupby(['country_name', 'game_year', 'game_season'])['medal_value'].sum().reset_index(name='country_total_medals_year')

# fusion
df_main = pd.merge(df_target, df_delegation, on=['country_name', 'discipline_title', 'game_year', 'game_season'], how='left')
df_main = pd.merge(df_main, df_sport_scale, on=['discipline_title', 'game_year', 'game_season'], how='left')
df_main = pd.merge(df_main, df_country_total, on=['country_name', 'game_year', 'game_season'], how='left')

df_main.head()

,country_name,discipline_title,game_year,game_season,is_host,medals_won,delegation_size,total_medals_available_in_sport,country_total_medals_year
0,Afghanistan,Athletics,1960,Summer,0,0,4,205,0
1,Afghanistan,Athletics,1996,Summer,0,0,1,264,0
2,Afghanistan,Boxing,1980,Summer,0,0,3,77,0
3,Afghanistan,Boxing,2004,Summer,0,0,1,77,0
4,Afghanistan,Boxing,2012,Summer,0,0,1,91,1


In [44]:
# historique sports


#   trie pour calculer les décalages
df_main = df_main.sort_values(by=['country_name', 'discipline_title', 'game_year'])

#  Historique dans ce sport precis (derniere fois quil a gagner, il a gagner combien)
df_main['prev_medals_sport'] = df_main.groupby(['country_name', 'discipline_title'])['medals_won'].shift(1).fillna(0)

#  Historique Puissance Pays (Combien le pays a gagné AU TOTAL aux derniers jeux ?)
# table temporaire pour faire le lag sur le total pays
df_country_lag = df_country_total.sort_values(['country_name', 'game_year'])
df_country_lag['prev_country_power'] = df_country_lag.groupby(['country_name'])['country_total_medals_year'].shift(1).fillna(0)

# fusion
df_main = pd.merge(df_main, df_country_lag[['country_name', 'game_year', 'prev_country_power']], on=['country_name', 'game_year'], how='left')

In [45]:
# 6. Sauvegarde
print("Aperçu des données nettoyées :")
df_main.head()

df_main.to_csv('olympic_data_clean.csv', index=False)
print("Fichier 'olympic_data_clean.csv' créé avec succès.")

Aperçu des données nettoyées :
Fichier 'olympic_data_clean.csv' créé avec succès.


In [46]:
df_main.head()

,country_name,discipline_title,game_year,game_season,is_host,medals_won,delegation_size,total_medals_available_in_sport,country_total_medals_year,prev_medals_sport,prev_country_power
0,Afghanistan,Athletics,1960,Summer,0,0,4,205,0,0.0,0.0
1,Afghanistan,Athletics,1996,Summer,0,0,1,264,0,0.0,0.0
2,Afghanistan,Boxing,1980,Summer,0,0,3,77,0,0.0,0.0
3,Afghanistan,Boxing,2004,Summer,0,0,1,77,0,0.0,0.0
4,Afghanistan,Boxing,2012,Summer,0,0,1,91,1,0.0,1.0


In [47]:
df_main.dtypes

,0
country_name,object
discipline_title,object
game_year,int64
game_season,object
is_host,int64
medals_won,int64
delegation_size,int64
total_medals_available_in_sport,int64
country_total_medals_year,int64
prev_medals_sport,float64
